## 蒙特卡洛搜索树
---
* 控制总搜索资源的量
* 在explore和exploit间做trad off

---
* 由于是随机对局，rollout次数较少时，反而没有alphabeta智能
  * minmax 搜索深度设为3，在gameState is_over的前3步，必能找出防止玩家获胜的策略
  * 相比较MCTS的随机rollout，500时结果不太靠谱。 但是提升到5000，开始有点智能的味道。
* Tie局面是否需要合并到win count中来辅助策略选择，保证不输。目前只看赢面的大小

todo 
  计算效率问题，用golang重写，可能1W或者5K的检索可以缩小到合理时间，看着还挺智能的。快10倍，1W次可以在1分钟左右处理完
  board size变大后，rollout耗时边长，但是tie率降低。目前只能设置到8

In [ ]:
# 环境配置
%cd ../../
import sys
sys.path.append('./python')

In [ ]:
from dlgobang import GameState, Player, Move, Point
from dlgobang.agent import MCTSAgent
from dlgobang.utils import print_board, print_move, point_from_coords

In [ ]:
def capture_diff(game_state):
    black_stones = 0
    white_stones = 0
    for r in range(1, game_state.board.num_rows + 1):
        for c in range(1, game_state.board.num_cols + 1):
            p = Point(r, c)
            color = game_state.board.get(p)
            if color == Player.BLACK:
                black_stones += 1
            elif color == Player.WHITE:
                white_stones += 1
    diff = black_stones - white_stones
    if game_state.next_player == Player.BLACK:
        return diff
    return -1 * diff

In [ ]:
BOARD_SIZE = 8
game = GameState.new_game(BOARD_SIZE)
bot = MCTSAgent(5000, temperature=1.4) # 围棋每一步需要搜索1W步才有业余1dan的效果

while not game.is_over():
    print_board(game.board)
    if game.next_player == Player.BLACK:
        human_move = input('输入落子[A1-]:')
        point = point_from_coords(human_move.strip())
        move = Move.play(point)
    else:
        move = bot.select_move(game)
    print_move(game.next_player, move)
    game = game.apply_move(move)